## Realizando a conexão com o MongoDB

In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# string de conexão
uri = "mongodb+srv://sheylacantalupo:@cluster-pipeline.ahm40a7.mongodb.net/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


Criando o banco de dados e uma coleção

In [3]:
db = client["db_temperaturas"]
collections_temp = db["temp"]
collections_all = db["tempAll"]

In [4]:
client.list_database_names()

['db_produtos', 'db_produtos_desafio', 'db_temperaturas', 'admin', 'local']

Inserindo um dado para teste

In [5]:
#tempTest = {"temperatura (°C)": "32"}
# collections.insert_one(tempTest)

In [6]:
client.list_database_names()

['db_produtos', 'db_produtos_desafio', 'db_temperaturas', 'admin', 'local']

## Extraindo os dados da API

In [7]:
import requests

api_key = ""
lat = -7.11532
lon =  -34.861
base_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

# Chamada à API usando o método GET
response = requests.get(base_url)

# Verifica se a requisição foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Acessando a reposta no formatato JSON:
    data = response.json()
    print(data)
else:
    # Se a requisição não foi bem-sucedida, imprime o código de status e o motivo
    print(f"Erro {response.status_code}: {response.reason}")


{'coord': {'lon': -34.8631, 'lat': -7.115}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 300.28, 'feels_like': 303.49, 'temp_min': 300.28, 'temp_max': 300.28, 'pressure': 1014, 'humidity': 83}, 'visibility': 10000, 'wind': {'speed': 0, 'deg': 0}, 'clouds': {'all': 0}, 'dt': 1708034663, 'sys': {'type': 1, 'id': 8391, 'country': 'BR', 'sunrise': 1707985427, 'sunset': 1708029821}, 'timezone': -10800, 'id': 3397277, 'name': 'João Pessoa', 'cod': 200}


## Inserindo as informações completas no db tempAll

In [8]:
collections_all.insert_one(data)

Extrair informações para compor o documento

In [9]:
from datetime import datetime

timestamp = data['dt']
data_hora = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S UTC')
temperatura_atual = data['main']['temp'] - 273.15
sensacao_termica = data['main']['feels_like'] - 273.15
temp_maxima = data['main']['temp_max'] - 273.15
temp_minima = data['main']['temp_min'] - 273.15

Criar documento a ser inserido

In [10]:
temp = {
    'data': data_hora.split()[0],
    'hora': data_hora.split()[1],
    'temperatura (°C)': temperatura_atual,
    'sensacao_termica (°C)': sensacao_termica,
    'temp_maxima (°C)': temp_maxima,
    'temp_minima (°C)': temp_minima
}

Inserindo um documento

In [11]:
collections_temp.insert_one(temp)

In [14]:
collections_temp.count_documents({})

9